In [ ]:
# Path to the CSV file in DBFS
path = "dbfs:/FileStore/tables/file.csv"

# Read the CSV file into a DataFrame with header option set to True
df = spark.read.format("csv").option("header", "True").load(path)

# Print the number of partitions before repartitioning
print("Before: {}".format(df.rdd.getNumPartitions()))

# Repartition the DataFrame into 5 partitions
df_part = df.repartition(5)

# Print the number of partitions after repartitioning
print("After: {}".format(df_part.rdd.getNumPartitions()))

# Write the original DataFrame to Parquet format in DBFS
df.write.format("parquet").mode("overwrite").save("dbfs:/FileStore/beforePartition/")

# Write the repartitioned DataFrame to Parquet format in DBFS
df_part.write.format("parquet").mode("overwrite").save("dbfs:/FileStore/afterPartition/")


Before: 1
After: 5


In [ ]:
# Function to display file names and sizes
def list_file_sizes(path):
    # List files and get their sizes
    files = dbutils.fs.ls(path)

    file_details = []
    for file_info in files:
        file_details.append((file_info.name, file_info.size))
    return file_details

# Path to the directory in DBFS before partitioning
path = "dbfs:/FileStore/beforePartition/"

# Get file details (original partition)
file_sizes = list_file_sizes(path)
display(file_sizes)

# Path to the directory in DBFS after partitioning
path = "dbfs:/FileStore/afterPartition/"

# Get file details (partitioned into 5 parts as per the example)
file_sizes = list_file_sizes(path)
display(file_sizes)

_1,_2
_SUCCESS,0
_committed_2153051212365741485,123
_started_2153051212365741485,0
part-00000-tid-2153051212365741485-1123a48a-0782-4ba6-9254-05469e844873-64-1-c000.snappy.parquet,2007


_1,_2
_SUCCESS,0
_committed_7821724679271796953,519
_started_7821724679271796953,0
part-00000-tid-7821724679271796953-765046ca-26e2-4fbd-97f0-c7ec961182f4-66-1-c000.snappy.parquet,1777
part-00001-tid-7821724679271796953-765046ca-26e2-4fbd-97f0-c7ec961182f4-67-1-c000.snappy.parquet,1813
part-00002-tid-7821724679271796953-765046ca-26e2-4fbd-97f0-c7ec961182f4-68-1-c000.snappy.parquet,1813
part-00003-tid-7821724679271796953-765046ca-26e2-4fbd-97f0-c7ec961182f4-69-1-c000.snappy.parquet,1813
part-00004-tid-7821724679271796953-765046ca-26e2-4fbd-97f0-c7ec961182f4-70-1-c000.snappy.parquet,1948


In [ ]:
# Partitioning by column:

df.repartition("column")

# Considerations for Choosing the Column for Partitioning

# Column Cardinality:
# Low Cardinality: Columns with few distinct values (low cardinality) are good candidates for partitioning because they result in a reasonable number of partitions.
# High Cardinality: Columns with many distinct values (high cardinality) can create too many partitions, which can be inefficient.

# Frequent Use in Filters:
# Choose columns that are frequently used in WHERE, FILTER, or JOIN clauses. This can improve performance because operations can benefit from partitioning.

# Data Distribution:
# Ensure that the chosen column results in a balanced distribution of data among the partitions. Imbalance can lead to "hotspots," where some partitions are much larger than others, causing performance bottlenecks.


In [ ]:
"""
Reduce shuffle.

Lazy evaluation practices (to give Spark more time to plan the execution), for example: create pointers to temporary views instead of moving data between tables.

Check Apache Spark logs!!!

Compatibility: check the system that will consume the data after processing and decide the best path. Below are some of the main databases and their best compatibilities:

Apache Hive
Ideal Format: Parquet
Why: Parquet is a columnar storage format optimized for efficient reading and data compression. It is highly compatible with Hive and improves the performance of analytical queries.

Amazon Redshift
Ideal Format: ORC, Parquet
Why: Both formats are columnar and optimized for fast and efficient analytics. They also integrate well with Redshift Spectrum for external queries.

Google BigQuery
Ideal Format: Avro, Parquet
Why: BigQuery is optimized to work with both columnar and row formats, but Parquet and Avro offer efficient compression and improved query performance.

Azure Synapse Analytics (formerly SQL Data Warehouse)
Ideal Format: Parquet
Why: Parquet offers excellent performance with Synapse Analytics due to its optimized columnar format.

Snowflake
Ideal Format: Parquet, ORC
Why: Snowflake is designed to work efficiently with columnar formats like Parquet and ORC, leveraging their compression and fast reading capabilities.

Apache Cassandra
Ideal Format: CQL (Cassandra Query Language)
Why: Cassandra is a NoSQL database that works best with its native CQL query format, which is optimized for fast writes and distributed reads.

MongoDB
Ideal Format: BSON (Binary JSON)
Why: BSON is MongoDB's native format, optimized for efficiently storing JSON documents, including support for rich data types.

Presto
Ideal Format: Parquet, ORC
Why: Presto is a distributed query engine optimized to work with columnar formats, offering fast performance for analytical queries.
"""